<a href="https://colab.research.google.com/github/hderoche/RNN_Bitcoin_Prediction/blob/master/RNN_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layer
from sklearn.preprocessing import MinMaxScaler

## Data Selection

In [5]:
data = pd.read_csv('Bitstamp_BTCUSD_1h.csv', header=0, sep=';')
data['diff'] = data['close'] - data['open']
data = data.drop(['unix', 'date', 'symbol', 'Volume USD', 'open'], axis=1)
print(data.head())

       high       low     close  Volume BTC    diff
0  21420.00  21244.04  21262.73  297.057663  -96.23
1  21576.76  21218.58  21365.70  639.172448  -27.85
2  21464.97  21203.77  21393.45  667.725259  186.95
3  21320.61  20740.55  21215.49  844.146644  385.38
4  20865.29  20758.58  20819.44  688.197022   49.33


Splitting the data into two parts

1.   Training data
2.   Testing data

The split_index is the percentage of the training data over the testing data


In [6]:
# In percentage
split_index = 70

index = int(np.floor(abs(data.shape[0] * split_index/100)))

data_train, data_test = data.loc[:index, :], data.loc[index + 1:, :]

print(data_train.shape)
print(data_test.shape)


(15906, 5)
(6816, 5)


In [7]:
scaler = MinMaxScaler(feature_range=(0,1))
data_training, data_testing = scaler.fit_transform(data_train), scaler.fit_transform(data_test)

## Getting the data for the train and test part
x_train = []
y_train = []

def data_selection(data_to_select):
    X = []
    y = []
    for i in range(data_to_select.shape[0] - 60):
        subset = []
        y.append(data_to_select[i + 60, 3])
        for j in range(60):
            subset.append(data_to_select[i + j])
        X.append(subset)
    X,y = np.array(X),np.array(y)

    return X, y

x_test = []
y_test = []
x_train = []
y_train = []

for i in range(60, data_train.shape[0] - 1):
    x_train.append(data_training[i - 60:i])
    y_train.append(data_training[i + 1, 3])

for i in range(60, data_test.shape[0] - 1):
    x_test.append(data_testing[i-60:i])
    y_test.append(data_testing[i + 1, 3])

x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)

print(x_test.shape)
print(x_train.shape)

(6755, 60, 5)
(15845, 60, 5)


#### Bot

In [64]:
class Bot():
  def __init__(self):
    self.nb_trade = 0
    self.nb_long = 0
    self.nb_short = 0
    self.open_position = 0
    self.price_on_entry = -1
    self.roe = []
    self.delta = []
    self.trade = []
  
  def long(self, data):
    if on_position():
      self.nb_trade += 1
      self.nb_long += 1
      self.open_position += 1
      self.price_on_entry = data['open']
    else:
      return 'Already on position'

  def short(self, data):
    if on_position():
      self.nb_trade += 1
      self.nb_short += 1
      self.open_position += 1
      self.price_on_entry = data['open']
    else:
      return 'Already on position'
  
  def close_long(self, data):
    if on_position() == False :
      self.open_position -= 1
      diff = data['open'] - self.price_on_entry
      self.delta.append(diff)
      ROE = (diff/self.price_on_entry)*100
      self.roe.append(ROE)
      t = Trade(self.nb_trade, diff, ROE)
      self.trade.append(t)
    else:
      return 'Not on position'
  
  def close_short(self, data):
    if on_position() == False :
      self.open_position -= 1
      diff = self.price_on_entry - data['open']
      self.delta.append(diff)
      ROE = (diff/self.price_on_entry)*100
      self.roe.append(ROE)
      t = Trade(self.nb_trade, diff, ROE)
      self.trade.append(t)
    else:
      return 'Not on position'
  
  def on_position(self):
    if self.open_position > 0:
      return True
    else:
      return False

  def add_trade(self, i):
    t = Trade(i, 100, 10)
    self.trade.append(t)

  def on_profit(self):
    if len(self.roe) > 0 and self.nb_trade > 0:
      mean_roe = sum(self.roe) / len(self.roe)
      if mean_roe > 0:
        return True, mean_roe
      else:
          return False, mean_roe
    else:
      print('No trade yet')
      return
  
  def show_trades(self):
    df_trade = pd.DataFrame.from_records([trade.to_dict() for trade in self.trade])
    return df_trade

class Trade():
  def __init__(self, nb_trade, delta, roe):
    self.nb_trade = nb_trade
    self.delta = delta
    self.roe = roe
  
  def to_string(self):
    print('Trade #' + str(self.nb_trade) + ' - Delta : ' + str(self.delta) + ' - PNL : ' + str(self.roe))
  
  def to_dict(self):
    return {
        '#': self.nb_trade,
        'Delta': self.delta,
        'PNL': self.roe
    }

## Defining the scoring system


Le bot de trading déterminera les ordres d'achat ou de vente si les conditions suivantes sont validées. La méthode de scoring permet de mettre une stratégie en place dès qu'un nombre définit de condition sont réunis

*   MA200 (donne la tendance générale du mouvement)
  *   Au dessus du cours -> Achat +1
  *   En dessous -> Vente -1
*   RSI
  *   Si en survente -> Achat +1
  *   En surachat -> Vente -1
*   Prédiction AI model
  * Si 1 -> +2
  * Si 0 -> -2

In [ ]:
# Initialisation du Bot
btcBot = Bot()

btcBot = Bot()
btcBot.add_trade(1)
btcBot.add_trade(2)
btcBot.add_trade(3)

df_trade = btcBot.show_trades()
print(df_trade)

In [10]:
# Data à instant t
def rules(data):
  score = 0
  if data['open'] > data['MA200'] :
    score += 1
  if data['open'] < data['MA200']:
    score -= 1
  if data['RSI'] > 70:
    score -= 1
  if data['RSI'] < 70:
    score += 1
  if data['pred'] == 1:
    score += 2
  if data['pred'] == 0:
    score -= 2
  return score

# Score to trigger an order
threshold = 2
# Target (in %) if reached, then close position
target = 10
def decision(score):
  if score > threshold:
    if btcBot.on_position() == False:
      btcBot.long()
  if score < - threshold:
    if btcBot.on_position() == False:
      btcBot.short()



### BackTesting

In [ ]:
def backtest():
  for i in range(data):
    

## AI Model

In [ ]:
def AI_model():
    model = tf.keras.Sequential()

    model.add(layer.LSTM(units = 60, activation=tf.nn.relu, return_sequences=True, input_shape= (x_train.shape[1], x_train.shape[2])))
    model.add(layer.Dropout(0.4))
    model.add(layer.LSTM(units = 60, activation=tf.nn.relu, return_sequences=True))
    model.add(layer.Dropout(0.2))
    model.add(layer.LSTM(units=60, activation=tf.nn.relu))
    model.add(layer.Dropout(0.2))
    model.add(layer.Dense(60))
    model.add(layer.Dense(6))


    # As this is a regression model, we want to calulate the price of the asset for the next day, we use the mean sqaured error as a loss funtion.
    # The adam optimizer is the standard in the AI libraries.
    model.compile(optimizer='adam',
                  loss = 'mean_squared_error')

     # Show how the RNN is structured and how many parameters there is.
    print(model.summary())

    # Fitting the model to the train dataset
    # Changing the epoch number will decrease the render time

    model.fit(x_train, y_train, epochs=10, batch_size=32)


    # Saving the model
    model.save('btcAI')

In [ ]:
AI_model()